In [1]:
from langchain_community.document_loaders import TextLoader, WebBaseLoader, PyPDFLoader,DirectoryLoader,UnstructuredMarkdownLoader
import bs4
import os

# from langchain_community.document_loaders import PyPDFDirectoryLoader
# loader = PyPDFDirectoryLoader('file/')
# txt = loader.load()
loader = DirectoryLoader(
    path="Z language",  # Replace with your folder path
    loader_cls=UnstructuredMarkdownLoader,       # Use TextLoader for .txt files
    use_multithreading=True      # Optional: speed up loading
)

txt = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.
Need to load profiles.
Need to load profiles.
Need to load profiles.
Need to load profiles.
Need to load profiles.
Need to load profiles.
Need to load profiles.


In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from pydantic.v1 import BaseModel, Field
import os

# Add this before your FAISS imports
class VectorStoreConfig(BaseModel):
    class Config:
        arbitrary_types_allowed = True


# Text splitting
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Characters (adjust for token limits)
    chunk_overlap=200,
    separators=[
        # ----- Markdown -----
        # Headers (regex: newline + #)
        r"\n# ", r"\n## ", r"\n### ", r"\n#### ",
        r"\n```\n",                                 # Code block boundaries
        # Horizontal rules (*** or ---)
        r"\n\*\*\*\n", r"\n---\n",

        # ----- LaTeX/Math -----
        r"\n\\begin{equation}",                     # LaTeX equations
        r"\n\\begin{align}",                        # Math align environments
        r"\n\\section{", r"\n\\subsection{",        # LaTeX sections
        r"\n\$\$", r"\n\\\[",                       # Display math boundaries

        # ----- C Code -----
        r"\n}\n",                                   # End of code blocks/functions
        # Comment separators (e.g., // ----)
        r"\n// -{4,}\n",
        r"\n#ifdef ", r"\n#endif",                  # Preprocessor directives
        r"\n}\n\n",                                 # Double newline after braces

        # ----- Assembly -----
        # Section directives (e.g., .section .text)
        r"\n\.section\s",
        r"\n\.global\s",                            # Global labels
        r"\n[a-zA-Z_]+:\n",                         # Labels (e.g., "main:\n")
        # Assembly comment separators (e.g., ; ----)
        r"\n; -{4,}\n",

        # ----- Generic Fallbacks -----
        r"\n\n", r"\n", r" ", r""                  # Standard text fallbacks
    ],
    is_separator_regex=True,   # Enable regex mode for all separators
    keep_separator=True,       # Retain separators in chunks (e.g., headers)
    # Clean trailing/leading whitespace (except in code)
    strip_whitespace=True,
)
fdocs = text_splitter.split_documents(txt)

# Embedding with CPU optimization
embeddings = SentenceTransformerEmbeddings(
    model_name="all-MiniLM-L12-v2",  
    model_kwargs={"device": "cpu"},
    encode_kwargs={
        "batch_size": 64,  # Process 64 chunks at once
        "normalize_embeddings": True  # Better for similarity
    }
)

# Create FAISS index (without persist_directory which is not supported)
db = FAISS.from_documents(
    documents=fdocs,
    embedding=embeddings,
    normalize_L2=True  # Reduces memory usage
)

# Save the FAISS index to disk (FAISS uses a different persistence method)
save_directory = "./faiss_index"
os.makedirs(save_directory, exist_ok=True)
db.save_local(save_directory)

# To load it back later, you would use:
# loaded_db = FAISS.load_local(save_directory, embeddings)

C:\Users\PC\AppData\Local\Temp\ipykernel_5552\578049552.py:57: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(
c:\Users\PC\OneDrive\Bureau\Masterpiece ESI\llm_rag_test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# !pip install faiss-cpu

In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, WebBaseLoader, PyPDFLoader, DirectoryLoader, UnstructuredMarkdownLoader
import bs4
import os

# Document loading
loader = DirectoryLoader(
    path="dd",  # Replace with your folder path
    loader_cls=UnstructuredMarkdownLoader,
    use_multithreading=True
)

txt = loader.load()
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_community.vectorstores import FAISS
from pydantic.v1 import BaseModel, Field
import os

# Add this before your FAISS imports
class VectorStoreConfig(BaseModel):
    class Config:
        arbitrary_types_allowed = True


# Text splitting
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=[
        # ----- Markdown -----
        r"\n# ", r"\n## ", r"\n### ", r"\n#### ",
        r"\n```\n",
        r"\n\*\*\*\n", r"\n---\n",

        # ----- LaTeX/Math -----
        r"\n\\begin{equation}",
        r"\n\\begin{align}",
        r"\n\\section{", r"\n\\subsection{",
        r"\n\$\$", r"\n\\\[",

        # ----- C Code -----
        r"\n}\n",
        r"\n// -{4,}\n",
        r"\n#ifdef ", r"\n#endif",
        r"\n}\n\n",

        # ----- Assembly -----
        r"\n\.section\s",
        r"\n\.global\s",
        r"\n[a-zA-Z_]+:\n",
        r"\n; -{4,}\n",

        # ----- Generic Fallbacks -----
        r"\n\n", r"\n", r" ", r""
    ],
    is_separator_regex=True,
    keep_separator=True,
    strip_whitespace=True,
)
fdocs = text_splitter.split_documents(txt)

# Embedding with CPU optimization
embeddings = SentenceTransformerEmbeddings(
    model_name="all-MiniLM-L12-v2",  
    model_kwargs={"device": "cpu"},
    encode_kwargs={
        "batch_size": 64,
        "normalize_embeddings": True
    }
)

# Create FAISS index
db = FAISS.from_documents(
    documents=fdocs,
    embedding=embeddings,
    normalize_L2=True
)

# Save the FAISS index to disk
save_directory = "./faiss_index"
os.makedirs(save_directory, exist_ok=True)
db.save_local(save_directory)


USER_AGENT environment variable not set, consider setting it to identify your requests.
Need to load profiles.
Need to load profiles.
Need to load profiles.
Need to load profiles.
Need to load profiles.
Need to load profiles.
Need to load profiles.
C:\Users\PC\AppData\Local\Temp\ipykernel_21044\2126956972.py:63: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(
c:\Users\PC\OneDrive\Bureau\Masterpiece ESI\llm_rag_test\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import

In [ ]:

# To load it back later:
# loaded_db = FAISS.load_local(save_directory, embeddings)

# Now let's modify to use GROQ API instead of Ollama
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_groq import ChatGroq  # Import the GROQ chat model
from langchain_core.prompts import ChatPromptTemplate
import os
# Load environment variables from .env file
load_dotenv()

# Set your GROQ API key from environment variable
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

# Initialize the GROQ LLM - replaced Ollama with GROQ
llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",  # You can choose "mixtral-8x7b-32768" or other models
    temperature=0,
    max_tokens=4096,  # Control response length
)

# The prompt template remains the same
# The prompt template with Z language focus
prompt = ChatPromptTemplate.from_template("""
[Context]
{context}
[/Context]

[Question]
{input}
[/Question]

**Instructions for Z Language Analysis (when question or context involves Z)**:

1.  **Identification**:
    *   Actively look for Z notation, schemas (state, operation, generic), schema calculus expressions (∧, ∨, ⇒, •, ;, ⨾, ∆, Ξ, etc.), types, sets, relations, functions, sequences, and predicates within the provided [Context].

2.  **Interpretation & Explanation**:
    *   If Z specifications are present in the [Context] and relevant to the [Question], explain their meaning in a clear, mathematically precise way.
    *   Describe the purpose of identified Z schemas: what state they define, what operations they specify, or what properties they encapsulate.
    *   Explain the pre-conditions and post-conditions of operation schemas.
    *   Clarify the meaning of logical predicates and invariants.

3.  **Representation & Generation (if requested or appropriate)**:
    *   If the [Question] asks to create or modify a Z specification, use standard Z notation.
    *   Present Z schemas clearly, typically using schema boxes with declarations above the line and predicates below.
    *   When discussing schema operations, clearly show inputs (e.g., `x? : X`), outputs (e.g., `y! : Y`), and state changes (e.g., `∆State`, `ΞState`).
    *   Ensure type consistency in any Z fragments you generate or analyze.

4.  **Z-Specific Response Structure (use this structure if the primary focus is Z)**:

    ## 1. Z Formal Analysis
    *   **Concept Definition**: Mathematical foundation of the Z concept(s) in question.
    *   **Notation Involved**: Key Z symbols and constructs relevant to the query (e.g., `∀`, `∃`, `ℙ`, `↔`, `≜`).

    ## 2. Z Specification Details
    *   **State Schemas**: Definition and explanation of relevant state schemas.
    *   **Invariants**: Discussion of state invariants.
    *   **Operation Schemas**: Breakdown of operation schemas, including:
        *   Declarations (inputs, outputs, state components)
        *   Pre-conditions
        *   Post-conditions

    ## 3. Schema Calculus & Logic
    *   Explanation of how schemas are combined or manipulated (if applicable).
    *   Analysis of logical predicates.

    ## 4. Verification & Refinement Aspects (if applicable)
    *   Proof obligations or consistency checks implied.
    *   Potential refinement steps or considerations.

**General Strict Instructions (apply to all answers, Z-specific or otherwise)**:
1.  **Language**:
    - Français avec des mots clés en Anglais.
    - Registre académique formel.

2.  **Content**:
    - Priorité absolue aux concepts théoriques du cours présentés dans le [Context].
    - Exercices UNIQUEMENT si :
        a) La [Question] le demande explicitement.
        b) Nécessaire pour illustrer une application critique.
    - Max 1 exemple/application par concept majeur.
    - Si la question concerne l'assembleur, c'est l'assembleur 8086, utiliser la même syntaxe que dans les documents du contexte, pas de registres 32 bits, uniquement 16 bits.

3.  **General Response Structure (use if Z is not the primary focus)**:
    ## Introduction
    - Contextualisation du sujet
    - Énoncé de l'objectif principal

    ## Développement Structuré
    ### Concepts Clés
    - Définitions formelles (théorèmes, formules)
    - Propriétés essentielles

    ### Explications Détaillées
    - Logique mathématique
    - Concepts importants

    ### Applications Ciblées
    [Si pertinent]
    - Exemple concret
    - Cas d'usage typique

    ## Conclusion Synthétique
    - Résumé en 3 points maximum
    - Liens avec d'autres concepts (si pertinent)
""")

# Create document chain
document_chain = create_stuff_documents_chain(llm, prompt)

# Create retrieval chain
retriever = db.as_retriever(
    search_kwargs={"k": 4}  # Retrieve top 4 most relevant documents
)

from langchain.chains import create_retrieval_chain

retriever_chain = create_retrieval_chain(retriever, document_chain)

# Example usage
def query_system(question):
    result = retriever_chain.invoke({'input': question})
    return result['answer']

# Example query
if __name__ == "__main__":
    from pprint import pprint
from dotenv import load_dotenv
    
    # Example query
    query = "donne un program assambleur pour entrer un nom et le convertir vers majiscule et l'afficher"
    response = query_system(query)
    
    print("\n===== RÉPONSE =====\n")
    print(response)


===== RÉPONSE =====

<think>
Bon, je dois écrire un programme en assembleur 8086 qui demande à l'utilisateur de saisir un nom, le convertit en majuscules, puis l'affiche. D'abord, je me souviens que pour lire du texte en assembleur, je dois utiliser des interruptions BIOS, comme l'interruption 0x21 avec AH=0x0E pour afficher des caractères, ou AH=0x0C pour lire un caractère.

Je dois donc commencer par afficher un message invitant l'utilisateur à entrer son nom. Ensuite, je dois lire chaque caractère saisi, les stocker en mémoire, et les convertir en majuscules. Pour cela, je peux utiliser la logique de masquage : soustraire 0x20 de chaque caractère lettres minuscules pour les convertir en majuscules.

Ensuite, une fois le nom saisi et converti, je dois l'afficher à l'écran. Je dois donc appeler à nouveau l'interruption 0x21 pour afficher chaque caractère du nom.

Je dois aussi gérer la fin de la saisie, probablement en attendant que l'utilisateur appuie sur la touche Entrée. Donc, je

: 

In [14]:
!pip freeze > requirements.txt